In [1]:

# If needed, uncomment:
# !pip install -U pip tensorflow==2.15.0 tensorflow-addons==0.22.0 matplotlib scikit-learn


In [2]:

import os, sys, json, math, random, time
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))



2.15.0
[]
2.15.0
[]


In [3]:

# ==== CONFIG ====
DATA_DIR = "dataset"                   # Your actual dataset directory
IMG_SIZE = 224                         # MobileNetV3 default sizes: 224 works well
BATCH_SIZE = 32
EPOCHS = 20
MODEL_VARIANT = "small"                # "small" or "large"

# Export/Artifacts
OUT_DIR = "diabetes_mbv3_artifacts"    # Local output directory
MODEL_NAME = "mbv3_diabetes_2class"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

print("Artifacts will be saved to:", OUT_DIR)


Artifacts will be saved to: diabetes_mbv3_artifacts


In [5]:

# ==== AUGMENTATION ====
data_augmentation = keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")


In [6]:

# ==== MODEL ====
if MODEL_VARIANT.lower() == "large":
    base = keras.applications.MobileNetV3Large(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet",
        include_preprocessing=False,
        pooling=None
    )
else:
    base = keras.applications.MobileNetV3Small(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet",
        include_preprocessing=False,
        pooling=None
    )

base.trainable = False  # start with transfer learning (frozen)

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input_image")
x = data_augmentation(inputs)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D(name="gap")(x)
x = layers.Dropout(0.2, name="dropout")(x)

# Keep 2-output structure (batch, 2) – softmax gives probabilities
outputs = layers.Dense(2, activation="softmax", name="predictions")(x)

model = keras.Model(inputs, outputs, name=f"mbv3_{MODEL_VARIANT}_diabetes_2class")
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)



Model: "mbv3_small_diabetes_2class"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 augmentation (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 MobilenetV3small (Function  (None, 7, 7, 576)         939120    
 al)                                                             
                                                                 
 gap (GlobalAveragePooling2  (None, 576)               0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 576)               0         
Model: "mbv3_small_diabetes_2class"
_________________________________________________________________
 La

In [7]:

# ==== CALLBACKS ====
ckpt_path = str(Path(OUT_DIR) / f"{MODEL_NAME}_best.keras")
callbacks = [
    keras.callbacks.ModelCheckpoint(
        ckpt_path, monitor="val_accuracy", save_best_only=True, save_weights_only=False
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy", patience=5, restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.2, patience=3, verbose=1
    )
]


In [8]:

# ==== TRAIN ====
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Unfreeze base for fine-tuning (optional)
base.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)
history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=max(5, EPOCHS//2),
    callbacks=callbacks
)


Epoch 1/20




79/79 [==============================] - 17s 173ms/step - loss: 0.6025 - accuracy: 0.6700 - val_loss: 0.5534 - val_accuracy: 0.6650 - lr: 0.0010
Epoch 2/20
79/79 [==============================] - 17s 173ms/step - loss: 0.6025 - accuracy: 0.6700 - val_loss: 0.5534 - val_accuracy: 0.6650 - lr: 0.0010
Epoch 2/20
79/79 [==============================] - 13s 162ms/step - loss: 0.4672 - accuracy: 0.7880 - val_loss: 0.4753 - val_accuracy: 0.7700 - lr: 0.0010
Epoch 3/20
79/79 [==============================] - 13s 162ms/step - loss: 0.4672 - accuracy: 0.7880 - val_loss: 0.4753 - val_accuracy: 0.7700 - lr: 0.0010
Epoch 3/20
79/79 [==============================] - 13s 167ms/step - loss: 0.4304 - accuracy: 0.8092 - val_loss: 0.4519 - val_accuracy: 0.8117 - lr: 0.0010
Epoch 4/20
79/79 [==============================] - 13s 167ms/step - loss: 0.4304 - accuracy: 0.8092 - val_loss: 0.4519 - val_accuracy: 0.8117 - lr: 0.0010
Epoch 4/20
79/79 [==============================] - 12s 155m

In [9]:

# ==== EVALUATE ====
def eval_and_report(ds, split_name="val"):
    y_true = []
    y_pred = []
    for batch, labels in ds.as_numpy_iterator():
        preds = model.predict(batch, verbose=0)
        y_true.extend(labels.tolist())
        y_pred.extend(np.argmax(preds, axis=1).tolist())
    print(f"=== {split_name.upper()} REPORT ===")
    print(classification_report(y_true, y_pred, target_names=class_names))
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

eval_and_report(val_ds, "val")
if test_ds is not None:
    eval_and_report(test_ds, "test")


=== VAL REPORT ===
              precision    recall  f1-score   support

    diabetes       0.92      0.89      0.91       300
 nondiabetes       0.89      0.93      0.91       300

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600

Confusion matrix:
[[267  33]
 [ 22 278]]
=== TEST REPORT ===
              precision    recall  f1-score   support

    diabetes       1.00      0.80      0.89        25
 nondiabetes       0.83      1.00      0.91        25

    accuracy                           0.90        50
   macro avg       0.92      0.90      0.90        50
weighted avg       0.92      0.90      0.90        50

Confusion matrix:
[[20  5]
 [ 0 25]]
=== TEST REPORT ===
              precision    recall  f1-score   support

    diabetes       1.00      0.80      0.89        25
 nondiabetes       0.83      1.00      0.91        25

    accuracy                           0.90  

In [10]:

# ==== SAVE SAVEDMODEL & LABELS ====
saved_model_dir = str(Path(OUT_DIR) / f"{MODEL_NAME}_SavedModel")
keras.models.save_model(model, saved_model_dir, include_optimizer=False)
print("SavedModel:", saved_model_dir)

labels_path = Path(OUT_DIR) / "labels.txt"
with open(labels_path, "w") as f:
    for name in class_names:
        f.write(name + "\n")
print("Labels saved to:", labels_path)


INFO:tensorflow:Assets written to: diabetes_mbv3_artifacts\mbv3_diabetes_2class_SavedModel\assets


INFO:tensorflow:Assets written to: diabetes_mbv3_artifacts\mbv3_diabetes_2class_SavedModel\assets


SavedModel: diabetes_mbv3_artifacts\mbv3_diabetes_2class_SavedModel
Labels saved to: diabetes_mbv3_artifacts\labels.txt


In [11]:

# ==== TFLITE CONVERSION ====

def make_representative_dataset(ds, num_batches=50):
    def rep():
        count = 0
        for batch, _ in ds.take(num_batches):
            batch = tf.cast(batch, tf.float32) / 255.0
            for img in batch:
                img = tf.expand_dims(img, 0)
                yield [img]
                count += 1
                if count >= num_batches:
                    return
    return rep

# FP32
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
tflite_path = str(Path(OUT_DIR) / f"{MODEL_NAME}_fp32.tflite")
with open(tflite_path, "wb") as f:
    f.write(tflite_model)
print("TFLite FP32:", tflite_path)

# FP16 (float16 weight quant)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_fp16 = converter.convert()
tflite_fp16_path = str(Path(OUT_DIR) / f"{MODEL_NAME}_fp16.tflite")
with open(tflite_fp16_path, "wb") as f:
    f.write(tflite_fp16)
print("TFLite FP16:", tflite_fp16_path)

# INT8 (full integer quant) – needs representative dataset
try:
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = make_representative_dataset(
        train_ds.unbatch().batch(1), num_batches=200
    )
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_int8 = converter.convert()
    tflite_int8_path = str(Path(OUT_DIR) / f"{MODEL_NAME}_int8.tflite")
    with open(tflite_int8_path, "wb") as f:
        f.write(tflite_int8)
    print("TFLite INT8:", tflite_int8_path)
except Exception as e:
    print("INT8 conversion skipped due to error:", e)


TFLite FP32: diabetes_mbv3_artifacts\mbv3_diabetes_2class_fp32.tflite
TFLite FP16: diabetes_mbv3_artifacts\mbv3_diabetes_2class_fp16.tflite
TFLite FP16: diabetes_mbv3_artifacts\mbv3_diabetes_2class_fp16.tflite
TFLite INT8: diabetes_mbv3_artifacts\mbv3_diabetes_2class_int8.tflite
TFLite INT8: diabetes_mbv3_artifacts\mbv3_diabetes_2class_int8.tflite


In [12]:

# ==== ANDROID/EDGE INFERENCE PARITY (TFLite Python Example) ====
# Example on a single image path (replace with your image to test output structure)
from PIL import Image

def preprocess_image_for_tflite(img_path, img_size=IMG_SIZE):
    img = Image.open(img_path).convert("RGB").resize((img_size, img_size))
    x = np.array(img).astype("float32") / 255.0
    x = np.expand_dims(x, axis=0)
    return x

def run_tflite_inference(tflite_file, input_array):
    interpreter = tf.lite.Interpreter(model_path=tflite_file)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # Assume single input, single output
    interpreter.set_tensor(input_details[0]["index"], input_array)
    interpreter.invoke()
    y = interpreter.get_tensor(output_details[0]["index"])
    return y

# Example usage (uncomment and set an image path):
# test_image_path = "/path/to/sample.jpg"
# x = preprocess_image_for_tflite(test_image_path)
# y = run_tflite_inference(tflite_fp16_path, x)
# print("TFLite output shape:", y.shape)  # Expect (1, 2)
# print("TFLite raw output:", y)
# print("Predicted class:", class_names[int(np.argmax(y, axis=1)[0])])



## Android (Kotlin) – Minimal TFLite Inference
Copy `*.tflite` and `labels.txt` to your app's assets. Then:

```kotlin
import org.tensorflow.lite.Interpreter
import android.content.res.AssetFileDescriptor
import java.nio.MappedByteBuffer
import java.nio.channels.FileChannel

fun loadModelFile(assetManager: android.content.res.AssetManager, modelPath: String): MappedByteBuffer {
    val fileDescriptor: AssetFileDescriptor = assetManager.openFd(modelPath)
    val inputStream = FileInputStream(fileDescriptor.fileDescriptor)
    val fileChannel = inputStream.channel
    val startOffset = fileDescriptor.startOffset
    val declaredLength = fileDescriptor.declaredLength
    return fileChannel.map(FileChannel.MapMode.READ_ONLY, startOffset, declaredLength)
}

// Assuming input is 224x224x3 float32 normalized to [0,1]
val interpreter = Interpreter(loadModelFile(assets, "mbv3_diabetes_2class_fp16.tflite"))
val inputShape = interpreter.getInputTensor(0).shape() // [1, 224, 224, 3]
val outputShape = interpreter.getOutputTensor(0).shape() // [1, 2]

val inputBuffer = ByteBuffer.allocateDirect(1 * 224 * 224 * 3 * 4).order(ByteOrder.nativeOrder())
// TODO: Fill inputBuffer with your preprocessed image data [0..1]

val outputBuffer = ByteBuffer.allocateDirect(1 * 2 * 4).order(ByteOrder.nativeOrder())
interpreter.run(inputBuffer, outputBuffer)

outputBuffer.rewind()
val probs = FloatArray(2)
outputBuffer.asFloatBuffer().get(probs)
val classes = assets.open("labels.txt").bufferedReader().readLines()
val predIdx = probs.indices.maxBy { probs[it] } ?: 0
val predLabel = classes[predIdx]
```

> The `.tflite` model outputs a 2-length vector per image (same output structure as training). Index-to-label is given by `labels.txt`.
